# Taller de búsqueda de hiperparámetros

En este taller vamos a explorar la búsqueda de hiperparámetros de manera automática.

## Nixon Lizcano

In [3]:
import time
import numpy as np

from sklearn.datasets import load_breast_cancer
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [4]:
X, y = load_breast_cancer(return_X_y=True)

In [5]:
test_size=0.25
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

**Ejemplo 1:** encuentra los mejores hiperparámetros usando `GridSearchCV`

In [6]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_grid = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza GridSearchCV
gs = GridSearchCV(clf, param_grid)

t0 = time.time()
gs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Tiempo de búsqueda: 0.472s


podemos ver que el atributo `cv_results_` nos entrega los resultados de toda la búsqueda.

In [7]:
dir(gs)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__firstlineno__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__sklearn_clone__',
 '__sklearn_tags__',
 '__static_attributes__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_check_refit_for_multimetric',
 '_check_scorers_accept_sample_weight',
 '_doc_link_module',
 '_doc_link_template',
 '_doc_link_url_param_generator',
 '_estimator_type',
 '_format_results',
 '_get_default_requests',
 '_get_doc_link',
 '_get_metadata_request',
 '_get_param_names',
 '_get_params_html',
 '_get_routed_params_for_fit',
 '_get_scorers',
 '_html_repr',
 '_parameter_constraints',
 '_repr_html_',
 '_repr

In [8]:
gs.cv_results_

{'mean_fit_time': array([0.0028367 , 0.002701  , 0.00253406, 0.00354619, 0.00346074,
        0.00338445, 0.00354981, 0.0033524 , 0.00330281, 0.0035584 ,
        0.00337086, 0.00329361, 0.00306373, 0.00304418, 0.00302939,
        0.00387735, 0.00381784, 0.00369658, 0.00380235, 0.00380359,
        0.00368099, 0.00387335, 0.00373197, 0.00368438]),
 'std_fit_time': array([2.42981777e-04, 1.76519255e-04, 8.24111468e-05, 3.51508441e-04,
        3.23666849e-04, 2.39734670e-04, 3.92576763e-04, 2.48985672e-04,
        2.45241203e-04, 3.87118390e-04, 2.20097289e-04, 2.33660941e-04,
        1.73449475e-05, 1.99021114e-05, 1.81789437e-05, 1.75769796e-04,
        1.61475495e-04, 5.70964662e-05, 7.57963224e-05, 1.45396446e-04,
        4.89833625e-05, 1.86793189e-04, 3.81407137e-05, 3.56945659e-05]),
 'mean_score_time': array([0.00040026, 0.00034037, 0.0003015 , 0.00029721, 0.00031452,
        0.00033808, 0.00029221, 0.0002923 , 0.00029559, 0.00030036,
        0.00029483, 0.00029073, 0.00029078, 0.00

Lo más importante es extraer los hiperparámetros del modelo que mejor error en de validación sacaron

In [9]:
gs.best_params_

{'criterion': 'entropy', 'max_depth': 18, 'min_samples_leaf': 1}

También es posible el mejor resultado en la métrica usada

In [10]:
gs.best_score_

np.float64(0.9389329685362517)

Finalmente, es posible extraer directamente un estimador que que ha sido creado con los mejores hiperparámetros.

In [11]:
gs.best_estimator_

,criterion,'entropy'
,splitter,'best'
,max_depth,18
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


**Ejemplo 2:** la clase `RandomizedSearchCV` se puede usar casi de la misma manera, solo que esta vez se debe escoger un número de combinaciones a evaluar; las cuales se escogeran de manera aleatoria.

In [12]:
clf = DecisionTreeClassifier()

# define los valores que usarás en la búsqueda del
# hiperparametro C
param_dist = {
    "max_depth": (3, 6, 12, 18),
    "min_samples_leaf": (1, 2, 3),
    "criterion": ["gini", "entropy"]
}

# Utiliza RandomizedSearchCV
rs = RandomizedSearchCV(clf, param_dist)

t0 = time.time()
rs.fit(X_train, y_train)
print("Tiempo de búsqueda: {:.3f}s".format(time.time() - t0))

Tiempo de búsqueda: 0.200s


In [13]:
rs.best_score_

np.float64(0.9366073871409029)

Podemos ver que aunque se demoró mucho menos, el resultado no es tan bueno.

## HyperOPt

In [14]:
%pip install hyperopt -q

Note: you may need to restart the kernel to use updated packages.


In [20]:
from hyperopt import hp, fmin, tpe, space_eval, STATUS_OK
from sklearn.model_selection import cross_val_score

In [22]:
def funcion_objetivo(params):
    clf = DecisionTreeClassifier(
        max_depth=int(params['max_depth']),
        min_samples_leaf=int(params['min_samples_leaf']),
        criterion=params['criterion']
    )

    scores = cross_val_score(clf, X_train, y_train,
                             cv=5,
                             scoring='accuracy')

    loss = 1.0 - np.mean(scores)
    return {'loss': loss, 'status': STATUS_OK}

params = {
    "max_depth": hp.choice("max_depth", (3, 6, 12, 18)),
    "min_samples_leaf": hp.choice("min_samples_leaf", (1, 2, 3)),
    "criterion": hp.choice("criterion", ["gini", "entropy"])
}

best = fmin(funcion_objetivo, params, algo=tpe.suggest, max_evals=100)

100%|██████████| 100/100 [00:02<00:00, 43.03trial/s, best loss: 0.061039671682626584]


In [23]:
best

{'criterion': np.int64(1),
 'max_depth': np.int64(3),
 'min_samples_leaf': np.int64(2)}